In [ ]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
# resnet50 is neural network model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
#re-sizing all the images
img_size=[224,224]

In [ ]:
test_path='/content/drive/MyDrive/Datasets/Test'
train_path='/content/drive/MyDrive/Datasets/Train'

In [ ]:
#import ResNet50 and add preprocessing layer
#here we will be using imagenet weights

resnet=ResNet50(input_shape=img_size+[3],weights='imagenet',include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
 #dont train existing weights
 for layer in resnet.layers:
   layer.trainable=False

In [ ]:
#useful for getting number of output classes
folders=glob('/content/drive/MyDrive/Datasets/Train/*')

In [ ]:
folders

['/content/drive/MyDrive/Datasets/Train/lamborghini',
 '/content/drive/MyDrive/Datasets/Train/mercedes',
 '/content/drive/MyDrive/Datasets/Train/audi']

In [ ]:
x=Flatten()(resnet.output)

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)
#create a model
model=Model(inputs=resnet.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
#tell the model what cost and optimization method to use 
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#use Image data generator for data augmantation
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True
                                 )
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                               )

Found 64 images belonging to 3 classes.


In [ ]:
test_set=test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Test',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                               )

Found 58 images belonging to 3 classes.


In [ ]:
#fit the model
r=model.fit(
    training_set,
    validation_data=test_set,
    epochs=50,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/50
2/2 [==============================] - 33s 14s/step - loss: 4.3363 - accuracy: 0.3125 - val_loss: 17.8221 - val_accuracy: 0.1552
Epoch 2/50
2/2 [==============================] - 2s 768ms/step - loss: 9.6504 - accuracy: 0.3281 - val_loss: 7.0056 - val_accuracy: 0.3276
Epoch 3/50
2/2 [==============================] - 1s 727ms/step - loss: 5.5990 - accuracy: 0.3906 - val_loss: 5.7514 - val_accuracy: 0.5172
Epoch 4/50
2/2 [==============================] - 1s 727ms/step - loss: 8.8687 - accuracy: 0.3125 - val_loss: 4.8623 - val_accuracy: 0.5172
Epoch 5/50
2/2 [==============================] - 1s 749ms/step - loss: 4.5571 - accuracy: 0.4219 - val_loss: 3.5474 - val_accuracy: 0.3276
Epoch 6/50
2/2 [==============================] - 1s 747ms/step - loss: 2.6264 - accuracy: 0.3906 - val_loss: 8.6299 - val_accuracy: 0.1552
Epoch 7/50
2/2 [==============================] - 1s 749ms/step - loss: 5.2568 - accuracy: 0.4375 - val_loss: 7.4568 - val_accuracy: 0.3793
Epoch 8/50
2/2 [====

In [ ]:
from tensorflow.keras.models import load_model
model.save('model_resnet50.h5')

In [ ]:
y_pred=model.predict(test_set)

2/2 [==============================] - 1s 123ms/step


In [ ]:
y_pred

array([[7.22730637e-01, 2.73637742e-01, 3.63163021e-03],
       [1.06543368e-02, 9.73250628e-01, 1.60950497e-02],
       [4.40106541e-03, 4.28695120e-02, 9.52729404e-01],
       [6.11075014e-02, 5.05580127e-01, 4.33312356e-01],
       [2.21757963e-01, 4.96825129e-01, 2.81416893e-01],
       [5.53872529e-03, 5.67824662e-01, 4.26636636e-01],
       [1.77225556e-05, 1.32506830e-04, 9.99849796e-01],
       [1.33784730e-02, 1.48493350e-01, 8.38128209e-01],
       [2.37730648e-02, 5.27621686e-01, 4.48605239e-01],
       [4.35604066e-01, 5.74205816e-02, 5.06975353e-01],
       [4.12918627e-02, 2.35198699e-02, 9.35188293e-01],
       [5.08119203e-02, 8.41851890e-01, 1.07336216e-01],
       [2.20664572e-02, 2.56416023e-01, 7.21517503e-01],
       [5.24301454e-02, 5.93589008e-01, 3.53980839e-01],
       [2.92239450e-02, 9.56914946e-03, 9.61206853e-01],
       [2.06493568e-02, 9.58168387e-01, 2.11822260e-02],
       [3.48690227e-02, 7.68937528e-01, 1.96193442e-01],
       [9.23084617e-02, 8.95561

In [ ]:
#taking index of maximum value in y_pred for each probability
import numpy as numpy 
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

array([0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 0, 0, 2, 1,
       1, 2, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 2, 0])

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/Datasets/Test/lamborghini/10.jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        ...,
        [ 13.,   2.,   0.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       [[ 17.,   7.,   0.],
        [ 17.,   7.,   0.],
        [ 18.,   8.,   0.],
        ...,
        [ 14.,   3.,   1.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       [[ 18.,   8.,   0.],
        [ 18.,   8.,   0.],
        [ 19.,   9.,   0.],
        ...,
        [ 14.,   3.,   1.],
        [ 14.,   4.,   3.],
        [ 14.,   4.,   3.]],

       ...,

       [[209., 129.,  92.],
        [210., 130.,  93.],
        [212., 132.,  95.],
        ...,
        [216., 132.,  96.],
        [214., 129.,  92.],
        [213., 129.,  92.]],

       [[208., 128.,  91.],
        [208., 128.,  91.],
        [211., 131.,  94.],
        ...,
        [215., 131.,  95.],
        [209., 129.,  94.],
        [207., 129.,  93.]],

       [[206., 128.,  90.],
        [206., 128.,  90.],
        [209., 1

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
x

array([[[0.06666667, 0.02745098, 0.        ],
        [0.06666667, 0.02745098, 0.        ],
        [0.06666667, 0.02745098, 0.        ],
        ...,
        [0.05098039, 0.00784314, 0.        ],
        [0.05490196, 0.01568628, 0.01176471],
        [0.05490196, 0.01568628, 0.01176471]],

       [[0.06666667, 0.02745098, 0.        ],
        [0.06666667, 0.02745098, 0.        ],
        [0.07058824, 0.03137255, 0.        ],
        ...,
        [0.05490196, 0.01176471, 0.00392157],
        [0.05490196, 0.01568628, 0.01176471],
        [0.05490196, 0.01568628, 0.01176471]],

       [[0.07058824, 0.03137255, 0.        ],
        [0.07058824, 0.03137255, 0.        ],
        [0.07450981, 0.03529412, 0.        ],
        ...,
        [0.05490196, 0.01176471, 0.00392157],
        [0.05490196, 0.01568628, 0.01176471],
        [0.05490196, 0.01568628, 0.01176471]],

       ...,

       [[0.81960785, 0.5058824 , 0.36078432],
        [0.8235294 , 0.50980395, 0.3647059 ],
        [0.83137256, 0

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 2s 2s/step


array([[0.00938716, 0.37499303, 0.61561984]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data),axis=1)

1/1 [==============================] - 0s 46ms/step


In [ ]:
a

array([2])

In [ ]:
i=image.load_img('/content/drive/MyDrive/Datasets/Train/audi/10.jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(i)

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 0s 29ms/step


array([[9.0425916e-04, 6.2457998e-03, 9.9284995e-01]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data),axis=1)

1/1 [==============================] - 0s 43ms/step


In [ ]:
a

array([2])